In [1]:
import pickle
import os
from bag_of_phrase_words import ChoraleSentences
from bag_of_phrases_utils import *
import torch

chorale_sentences_by_feature_combination_pickle_filename = \
    'chorale_sentences_by_feature_combination.pkl'
bags_of_phrases_directory_name = 'bags_of_phrases'

if chorale_sentences_by_feature_combination_pickle_filename not in os.listdir(os.getcwd()):
    # This is the first time we make the files, let it run.
    feature_combinations = make_feature_combinations()
    make_bags_of_phrases_from_feature_combinations(
        feature_combinations,
        bags_of_phrases_directory_name
    )
    make_chorale_sentences_by_feature_combination(
        chorale_sentences_by_feature_combination_pickle_filename,
        bags_of_phrases_directory_name
    )

with open(chorale_sentences_by_feature_combination_pickle_filename, 'rb') \
        as chorale_sentences_by_feature_combination_file:
    chorale_sentences_by_feature_combination = pickle.load(chorale_sentences_by_feature_combination_file)

basic_chorale_sentences : ChoraleSentences = chorale_sentences_by_feature_combination['basic_two_harmonies']

In [20]:
# This is the chorale sentences object which is our processed database
chorale_sentences = basic_chorale_sentences

# If we train with batches, we'll need all of our phrases to be of the same length (with padding to the shorter phrases
train_with_batches = True

# For convenient debug, we'll have an option to train on a single example (or just a few) and test on these examples
debug = True

hyperparams = dict(
    h_dim=1000,
    n_layers=10,
    dropout=0,
    batch_size=1,
    learn_rate=0.00001,
    lr_sched_factor=0.01,
    lr_sched_patience=1,
    early_stopping=100
)

In [21]:
from chorale_rnn import *
from trainer import RNNTrainer

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

print('hyperparams:\n', hyperparams)

# Dataset definition
vocab_len = chorale_sentences.vocabulary_length

batch_size = hyperparams['batch_size'] if train_with_batches else 1

train_test_ratio = 0.9
num_samples = chorale_sentences.number_of_chorales
num_train = int(train_test_ratio * num_samples)

if debug:
    train_index_start = 0
    train_index_end = 1
    test_index_start = 0
    test_index_end = 1
else:
    train_index_start = 0
    train_index_end = num_train
    test_index_start = num_train
    test_index_end = num_samples

samples, labels = chorale_sentences_to_labelled_samples(chorale_sentences, device)

ds_train = torch.utils.data.TensorDataset(
    samples[train_index_start:train_index_end], labels[train_index_start:train_index_end]
)
ds_test = torch.utils.data.TensorDataset(
    samples[test_index_start:test_index_end], labels[test_index_start:test_index_end]
)

dl_train = torch.utils.data.DataLoader(ds_train, batch_size, shuffle=False, drop_last=True)
dl_test = torch.utils.data.DataLoader(ds_test, batch_size, shuffle=False, drop_last=True)

print(f'Train: {len(dl_train):3d} batches,')
print(f'Test:  {len(dl_test):3d} batches,')

# Training definition
in_dim = out_dim = vocab_len
checkpoint_file = 'checkpoints/rnn'
num_epochs = 500
early_stopping = hyperparams['early_stopping']

model = MySequenceModel(
    nn.GRU,
    in_dim,
    hyperparams['h_dim'],
    out_dim,
    hyperparams['n_layers'],
    hyperparams['dropout']
)

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams['learn_rate'])
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='max',
    factor=hyperparams['lr_sched_factor'],
    patience=hyperparams['lr_sched_patience'],
    verbose=True
)

print(model)

trainer = RNNTrainer(model, loss_fn, optimizer, device)

Using device: cpu
hyperparams:
 {'h_dim': 1000, 'n_layers': 10, 'dropout': 0, 'batch_size': 1, 'learn_rate': 1e-05, 'lr_sched_factor': 0.01, 'lr_sched_patience': 1, 'early_stopping': 100}
Train:   1 batches,
Test:    1 batches,
MySequenceModel(
  (seq_model): GRU(27, 1000, num_layers=10, batch_first=True)
  (out_layer): Linear(in_features=1000, out_features=27, bias=True)
)


In [22]:
def post_epoch_fn(epoch, train_res, test_res, verbose):
    # Update learning rate
    scheduler.step(test_res.accuracy)
    # Sample from model to show progress
    if verbose:
        generated_sequence = generate_from_model(
            model, chorale_sentences, T=0.5
        )
        print(generated_sequence)

# Train, unless final checkpoint is found
checkpoint_file_final = f'{checkpoint_file}_final.pt'
if os.path.isfile(checkpoint_file_final):
    print(f'*** Loading final checkpoint file {checkpoint_file_final} instead of training')
    saved_state = torch.load(checkpoint_file_final, map_location=device)
    model.load_state_dict(saved_state['model_state'])
else:
    try:
        # Print pre-training sampling
        print(generate_from_model(model, chorale_sentences, T=0.5))

        fit_res = trainer.fit(dl_train, dl_test, num_epochs, max_batches=None,
                              post_epoch_fn=post_epoch_fn, early_stopping=early_stopping,
                              checkpoints=checkpoint_file, print_every=1)

    except KeyboardInterrupt as e:
        print('\n *** Training interrupted by user')


 *** Training interrupted by user
[25, 25, 0, 23, 17, 6, 26]
--- EPOCH 1/500 ---


train_batch:   0%|          | 0/1 [00:00<?, ?it/s]

test_batch:   0%|          | 0/1 [00:00<?, ?it/s]


*** Saved checkpoint checkpoints/rnn.pt
[25, 20, 16, 26]
--- EPOCH 2/500 ---


train_batch:   0%|          | 0/1 [00:00<?, ?it/s]

...

train_batch:   0%|          | 0/1 [00:00<?, ?it/s]

test_batch:   0%|          | 0/1 [00:00<?, ?it/s]

[25, 18, 5, 7, 23, 20, 11, 18, 6, 12, 24, 8, 3, 15, 13, 8, 25, 6, 8, 25, 17, 24, 7, 14, 22, 21, 26]
--- EPOCH 103/500 ---


train_batch:   0%|          | 0/1 [00:00<?, ?it/s]

test_batch:   0%|          | 0/1 [00:00<?, ?it/s]